## Crank-Slider Mechanism

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
me.init_vprinting()

In [ ]:
q1, q2, q3 = me.dynamicsymbols("q1, q2, q3")
q1d, q2d, q3d = me.dynamicsymbols("q1, q2, q3", 1)
u1, u2, u3 = me.dynamicsymbols("u1, u2, u3")
L1, L2 = symbols("L1, L2")

N, A, B = symbols("N, A, B", cls=me.ReferenceFrame)
A.orient_axis(N, N.z, q1)
B.orient_axis(A, A.z, q2 - q1)

O, P, Q, R, G1, G2 = symbols("O, P, Q, R, G1, G2", cls=me.Point)
P.set_pos(O, L1 * A.x)
Q.set_pos(P, L2 * B.x)
R.set_pos(O, q3 * N.x)
G1.set_pos(O, L1 / 2 * A.x)
G2.set_pos(P, L2 / 2 * B.x)

O.set_vel(N, 0)
P.v2pt_theory(O, N, A)
G1.v2pt_theory(O, N, A)
Q.v2pt_theory(P, N, B)
G2.v2pt_theory(P, N, B)
R.set_vel(N, u3 * N.x)

In [ ]:
t = symbols("t")
# configuration constraints: OR + (QP + QO) = 0
zero = R.pos_from(O) + O.pos_from(Q)
f_c = ImmutableDenseMatrix([zero & N.x, zero & N.y])
display(zero, f_c.simplify())

In [ ]:
# velocity constraints
dzero = zero.diff(t, N)
f_v = ImmutableDenseMatrix([dzero & N.x, dzero & N.y])
display(f_v.simplify())

In [ ]:
q_ind = [q1]
u_ind = [u1]
q_dep = [q2, q3]
u_dep = [u2, u3]
kde = [q1d - u1, q2d - u2, q3d - u3]

kane = me.KanesMethod(
    N,
    q_ind=q_ind,
    u_ind=u_ind,
    q_dependent=q_dep,
    u_dependent=u_dep,
    kd_eqs=kde,
    configuration_constraints=f_c,
    velocity_constraints=f_v
)

In [ ]:
g, m_crank, m_rod, m_slider = symbols("g, m_c, m_r, m_s")
id_crank = me.inertia(A, 0, 0, m_crank * L1**2 / 12)
id_rod = me.inertia(B, 0, 0, m_rod * L2**2 / 12)
inertia_crank = (id_crank, G1)
inertia_rod = (id_rod, G2)

crank = me.RigidBody("crank", G1, A, m_crank, inertia_crank)
rod = me.RigidBody("rod", G2, B, m_rod, inertia_rod)
slider = me.Particle("slider", R, m_slider)
bodies = [crank, rod, slider]

loads = [
    (G1, -m_crank * g * N.y),
    (G2, -m_rod * g * N.y),
    (R, -m_slider * g * N.y)
]

fr, frstar = kane.kanes_equations(bodies, loads)

In [ ]:
from spb import *

constants = {
    m_crank: 5,
    m_rod: 10,
    m_slider: 1,
    L1: 1,
    L2: 2.5,
    g: 9.81
}

q2q3_eq = [c.subs(constants).subs(q1, pi/4) for c in f_c]
plot_contour(*q2q3_eq, (q2, 0, 2*pi), (q3, 0, 5), {"levels": [0]}, is_filled=False)

In [ ]:
res = nsolve(q2q3_eq, [q2, q3], (6, 3))
res

In [ ]:
import numpy as np
from pydy.system import System

sys = System(kane)
sys.constants = constants
sys.initial_conditions = {
    q1: pi / 4, q2: 5.99642875496804, q3: 3.10502254284291,
    u1: 0, u2: 0, u3: 0
}

fps = 60
t0, tf = 0, 15
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)

results = sys.integrate()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(sys.times, results[:, :3])
ax[0].set_ylabel(r"Coordinates")
ax[0].legend(["$q_{1}$", "$q_{2}$", "$q_{3}$"])
ax[1].plot(sys.times, results[:, 3:])
ax[1].legend(["$u_{1}$", "$u_{2}$", "$u_{3}$"])
ax[1].set_ylabel(r"Velocities")
ax[1].set_xlabel("Time [s]")
plt.show()

In [ ]:
from pydy.viz import Scene, Cylinder, Cube, VisualizationFrame

crank_geom = Cylinder(length=L1, radius=0.05, color='grey', name="crank")
rod_geom = Cylinder(length=L2, radius=0.05, color='grey', name="rod")
slider_geom = Cube(0.2, color="orange", name="slider")

crank_frame, rod_frame = symbols("crank_frame, rod_frame", cls=me.ReferenceFrame)
crank_frame.orient_axis(A, A.z, -pi / 2)
rod_frame.orient_axis(B, B.z, -pi / 2)
crank_vf = VisualizationFrame('crank vf', crank_frame, G1, crank_geom)
rod_vf = VisualizationFrame('rod vf', rod_frame, G2, rod_geom)
slider_vf = VisualizationFrame('slider vf', N, R, slider_geom)

scene = Scene(N, O, crank_vf, rod_vf, slider_vf, system=sys)
scene.display_jupyter(axes_arrow_length=1)

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def _crank(y, idx, r):
    q1 = y[idx, 0]
    x = [0, r * np.cos(q1)]
    y = [0, r * np.sin(q1)]
    return x, y

def _rod(y, idx, r, L):
    q1 = y[idx, 0]
    q2 = y[idx, 1]
    x = [r * np.cos(q1), r * np.cos(q1) + L * np.cos(q2)]
    y = [r * np.sin(q1), r * np.sin(q1) + L * np.sin(q2)]
    return x, y

idx = 0
fig, ax = plt.subplots()
ax.axhline(0, linestyle=":", color="darkgray")
crank, = ax.plot(*_crank(results, idx, constants[L1]), label="crank")
rod, = ax.plot(*_rod(results, idx, constants[L1], constants[L2]), label="rod")
slider = ax.scatter(results[idx, 2], 0, color="r", label="slider")
ax.legend()
ax.set_aspect("equal")
ax.axis([-1.5, 4, -2, 2])

def update(idx):
    crank.set_data(*_crank(results, idx, constants[L1]))
    rod.set_data(*_rod(results, idx, constants[L1], constants[L2]))
    slider.set_offsets([results[idx, 2], 0])
    ax.set_title("t = {:.2f} s".format(sys.times[idx]))

ani = FuncAnimation(fig, update, frames=len(sys.times))
HTML(ani.to_jshtml(fps=fps))

## With the Joints Framework

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
import numpy as np
from pydy.system import System
from pydy.viz import Scene, Cylinder, Cube, VisualizationFrame
me.init_vprinting()

q1, q2, q3 = me.dynamicsymbols("q1, q2, q3")
u1, u2, u3 = me.dynamicsymbols("u1, u2, u3")
L1, L2, m_crank, m_rod, m_slider, g = symbols("L1, L2, m_c, m_r, m_s, g")
t = symbols("t")

N, A, B, S = symbols("N, A, B, S", cls=me.ReferenceFrame)

In [ ]:
wall = me.Body("Wall", frame=N)
crank = me.Body(
    "Crank", frame=A, mass=m_crank,
    central_inertia=me.inertia(A, 0, 0, m_crank * L1 ** 2 / 12))
rod = me.Body(
    "Rod", frame=B, mass=m_rod,
    central_inertia=me.inertia(A, 0, 0, m_rod * L2 ** 2 / 12))
slider = me.Body("Slider", frame=S, mass=m_slider)

crank.apply_force(-m_crank * g * N.y)
rod.apply_force(-m_rod * g * N.y)
slider.apply_force(-m_slider * g * N.y)

In [ ]:
j1 = me.PinJoint(
    "j1", wall, crank, coordinates=q1, speeds=u1,
    child_point=-L1 / 2 * crank.x,
    joint_axis=wall.z
)
j2 = me.PinJoint(
    "j2", crank, rod, coordinates=q2, speeds=u2,
    parent_point=L1 / 2 * crank.x,
    child_point=-L2 / 2 * rod.x,
    joint_axis=crank.z
)
j3 = me.PrismaticJoint(
    "j3", wall, slider, coordinates=q3, speeds=u3
)

zero = rod.masscenter.pos_from(slider.masscenter) + L2 / 2 * rod.x
f_c = Matrix([zero & N.x, zero & N.y])
f_c.simplify()
f_v = f_c.diff(t)
f_v.simplify()
display(zero, f_c, f_v)

In [ ]:
method = me.JointsMethod(wall, j1, j2, j3)

kane = me.KanesMethod(
    wall.frame,
    q_ind=[q1],
    u_ind=[u1],
    q_dependent=[q2, q3],
    u_dependent=[u2, u3],
    kd_eqs=method.kdes,
    configuration_constraints=f_c,
    velocity_constraints=f_v,
    forcelist=method.loads,
    bodies=method.bodies)

fr, frstar = kane.kanes_equations()

In [ ]:
sys = System(kane)
sys.constants = {
    m_crank: 5,
    m_rod: 10,
    m_slider: 1,
    L1: 1,
    L2: 2.5,
    g: 9.81
}
sys.initial_conditions = {
    q1: pi / 4, q2: 5.21103059157059, q3: 3.10502254284291,
    u1: 0, u2: 0, u3: 0
}
fps = 60
t0, tf = 0, 15
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)
results = sys.integrate()

crank_geom = Cylinder(length=L1, radius=0.05, color='grey', name="crank")
rod_geom = Cylinder(length=L2, radius=0.05, color='grey', name="rod")
slider_geom = Cube(0.2, color="orange", name="slider")

crank_frame, rod_frame = symbols("crank_frame, rod_frame", cls=me.ReferenceFrame)
crank_frame.orient_axis(A, A.z, -pi / 2)
rod_frame.orient_axis(B, B.z, -pi / 2)

crank_vf = VisualizationFrame('crank vf', crank_frame, crank.masscenter, crank_geom)
rod_vf = VisualizationFrame('rod vf', rod_frame, rod.masscenter, rod_geom)
slider_vf = VisualizationFrame('slider vf', N, slider.masscenter, slider_geom)

# arguments to Scene: reference frame, origin, visual frames, system
O = crank.masscenter.locatenew("O", -L1/2 * crank.frame.x)
scene = Scene(N, O, crank_vf, rod_vf, slider_vf, system=sys)
scene.display_jupyter(axes_arrow_length=5)